In [1]:
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import Point
from rasterio.warp import transform
from pyproj import CRS
from pyproj import Transformer
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)

In [2]:
trends = pd.read_csv('merge_data/trends_clean.csv')
trends_filtered = trends[['site_id','latitude', 'longitude']]

#split into chunks for faster processing.
trends_filtered_1 = trends_filtered.iloc[:3500,:].reset_index(drop = True)
trends_filtered_2 = trends_filtered.iloc[3500:7000,:].reset_index(drop = True)
trends_filtered_3 = trends_filtered.iloc[7000:10500,:].reset_index(drop = True)
trends_filtered_4 = trends_filtered.iloc[10500:14000,:].reset_index(drop = True)
trends_filtered_5 = trends_filtered.iloc[14000:18000,:].reset_index(drop = True)
trends_filtered_6 = trends_filtered.iloc[18000:,:].reset_index(drop = True)

## Process all geotifs to read the evapostranspiration data

In [3]:
def read_tifs(df):
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    years = [str(x) for x in range(2000,2019)]
    all_dicts = {}
    for site in df['site_id']:
        all_dicts[site] = {'site':[site]*19, 'year':years}
        for month in months:
            all_dicts[site][month] = []
    
    for year in years:
        print('******')
        print('******')
        print('******')
        print(year)
        print('------')
        month_vals = []                
        for month in months:
            print(month)
            
            year_month = year + '_' + month
            column = 'AET_' + year_month
            
            if column == 'AET_2018_10':
                for m in ['10', '11', '12']:
                    for i, row in df.iterrows():
                        all_dicts[row['site_id']][m].append(None)
                break

            file_path = 'monthly_ets/AET_' + year_month + '.tif'

            count = 0
            with rasterio.open(file_path) as src:
                #set source and target coordinate systems
                src_crs = CRS("WGS84")
                dst_crs = CRS("NAD83")
                
                #pull the ET data from the geotifs for the site latitude and longitude
                for i, row in df.iterrows():
                    lat = row['latitude']
                    lon = row['longitude']
                    transformer = Transformer.from_crs(src_crs, dst_crs)
                    dst_lon, dst_lat = transformer.transform(lon, lat)
                    value = next(src.sample([(dst_lon, dst_lat)]))[0]
                    all_dicts[row['site_id']][month].append(value)

    return all_dicts


 Need to run below for each chunk, change output csv name each time

In [ ]:
full = read_tifs(trends_filtered_1) #change for each chunk

In [ ]:
all_dfs = []
for dictionary in full.values():
    df = pd.DataFrame.from_dict(dictionary, orient = 'columns')
    all_dfs.append(df)
out_df = pd.concat(all_dfs).reset_index(drop = True)

out_df.to_csv('ET_chunk1.csv', index = False) #change export name for each chunk

#### Can start from here if all processing of he geotifs has been completed: Merge chunks

In [6]:
chunk1 = pd.read_csv('ET_chunk1.csv')
chunk2 = pd.read_csv('ET_chunk2.csv')
chunk3 = pd.read_csv('ET_chunk3.csv')
chunk4 = pd.read_csv('ET_chunk4.csv')
chunk5 = pd.read_csv('ET_chunk5.csv')
chunk6 = pd.read_csv('ET_chunk6.csv')

et_df = pd.concat([chunk1, chunk2, chunk3, chunk4, chunk5, chunk6])
et_df = et_df.drop(['Unnamed: 0'], axis = 1)

In [7]:
et_df = et_df.rename(columns = {'01':'jan', '02':'feb', '03':'mar', '04':'apr', '05':'may', '06':'jun', 
                                '07':'jul', '08':'aug', '09':'sep', '10':'oct', '11':'nov', '12':'dec'
                               }
                    )

In [8]:
#we replace zeros with np.nan, since zeros are actually missing values
et_df.replace(0, np.nan, inplace=True)

#### Calculate the means and slopes for each month for each site.

In [ ]:
cleaned_dfs = {}
column_months = ['jan_mean', 'feb_mean', 'mar_mean', 'apr_mean', 'may_mean', 'jun_mean', 'jul_mean', 'aug_mean', 
                 'sep_mean', 'oct_mean', 'nov_mean', 'dec_mean', 'jan_slope', 'feb_slope', 'mar_slope', 'apr_slope', 
                 'may_slope', 'jun_slope', 'jul_slope', 'aug_slope', 'sep_slope', 'oct_slope', 'nov_slope', 'dec_slope'
                ]
column_months_spec = [x + '_ET' for x in column_months]

dropped = et_df.drop(['year'], axis = 1)
grouped = dropped.groupby(['site'])
means_df = grouped.mean()
groups = grouped.groups.keys()

et_dict = {}
count =0
for g in groups:
    group_df = grouped.get_group(g).reset_index(drop = True)
    count +=1
    if count %100 == 0:
        print(count)
        
    #pull mean data
    month_means = list(means_df.loc[g])

    #find slopes
    month_slopes = []
    for month in group_df.columns[1:]:
        data = group_df[month].dropna()
        if data.empty == True:
            month_slopes.append(np.nan)
            continue
        x = np.array(list(range(len(data)))).reshape(-1,1)
        y = np.array(data).reshape(-1,1)
        linreg = LinearRegression().fit(x, y)            
        slope = linreg.coef_[0][0]
        month_slopes.append(slope)

    #dict with list data for each site
    full_data = [g] + month_means + month_slopes
    et_dict[g] = full_data

In [10]:
et_cleaned = pd.DataFrame.from_dict(et_dict, orient = 'index')
et_cleaned.columns = ['site_id'] + column_months_spec
et_cleaned.reset_index(drop = True, inplace = True)

Now we handle missing data. Lets take the mean value from the two surrounding months.

In [11]:
cols = et_cleaned.columns
for i, row in et_cleaned.iterrows():
    count = 0
    for col in cols[1:13]:
        count+=1
        if pd.isna(row[col]) == True:
            if col == 'jan_mean':
                newval_mean = (row['feb_mean'] + row['dec_mean'])/2
                et_cleaned.loc[i, 'jan_mean'] = newval_mean
                newval_slope = (row['feb_slope'] + row['dec_slope'])/2
                et_cleaned.loc[i,'jan_mean'] = newval_slope
                continue
                
            if col == 'dec_mean':
                newval_mean = (row['nov_mean'] + row['jan_mean'])/2
                et_cleaned.loc[i, 'dec_mean'] = newval_mean
                newval_slope = (row['nov_slope'] + row['jan_slope'])/2
                et_cleaned.loc[i,'dec_slope'] = newval_slope
                continue
                                
            if pd.isna(row[cols[count-1]]) == True:
                newval_mean = row[cols[count+1]]
                newval_slope = row[cols[count+13]]
                et_cleaned.loc[i, col] = newval_mean
                et_cleaned.loc[i,cols[count +12]] = newval_slope
                continue
            if pd.isna(row[cols[count+1]]) == True:
                newval_mean = row[cols[count-1]]
                newval_slope = row[cols[count+11]]
                et_cleaned.loc[i, col] = newval_mean
                et_cleaned.loc[i,cols[count +12]] = newval_slope
                continue
                
                
            newval_mean = (row[cols[count-1]] + row[cols[count+1]])/2
            newval_slope = (row[cols[count+11]] + row[cols[count+13]])/2
            et_cleaned.loc[i, col] = newval_mean
            et_cleaned.loc[i,cols[count +12]] = newval_slope

In [13]:
et_cleaned.to_csv('merge_data/et_cleaned.csv', index = False)